<a href="https://colab.research.google.com/github/luuleitner/xMason/blob/main/xmasonsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# xMason
#### An adapted Mason model to calculate impedance responses of ultrasonic transducers

## Setup

In [ ]:
# xMason can be directly installed from the GitHub repository using PIP.

!pip install git+https://github.com/luuleitner/xMason

In [ ]:
# We need to import necessary packages into Colab. Don't worry to much about this cell of code

from simulation.src.data.loader import VNA_Dataloader
from simulation.src.models.simulate import simulate_xMason
from simulation.src.features.ports import frequency_spectrum

import os
import numpy as np # array handeling
from urllib.request import urlretrieve # for downloading the sample data
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.signal import find_peaks

## Load dataset

In [ ]:
# Import the material data and the ISAF22 VNA dataset of 6 printed P(VDF-TrFE) transducers

mason_path = '/content/xMasonData'
mat_path = os.path.join(mason_path, 'materials.csv')
ISAF22_VNAexperiments = os.path.join(mason_path, '/ISAFdata')

if os.path.exists(mason_path) == False:
  os.mkdir(mason_path)
  os.chdir(mason_path)
  !wget https://raw.githubusercontent.com/luuleitner/xMason/main/data/materials.csv

if os.path.exists(ISAF22_VNAexperiments) == False:
  os.mkdir(ISAF22_VNAexperiments)
  os.chdir(ISAF22_VNAexperiments)
  !wget -i https://raw.githubusercontent.com/luuleitner/xMason/main/data/dataset_ISAF22/isaf_dataset_url.txt

os.chdir(mason_path)

## Transducer stack

In [ ]:
# Define sizes and materials of the transducer stack

## Transducer stack sizes:
frequency_band = [20, 100]
transducer_radius = 0.0088

transducer_thickness_1 = 12.3 * 10 ** -6
transducer_thickness_2 = 16.6 * 10 ** -6

electrode_thickness = 0.5 * 10 ** -6
substrate_thickness = 13 * 10 ** -6


## Transducer stack materials:
top_load = bottom_load = 'Air'
electrode_mat = 'Silverink'
piezo_mat = 'P(VDF-TrFE)'
substrate_mat = 'Kapton'

## Simulation

In [ ]:
# Start simulation

Simulation_1 = simulate_xMason(parameters={'fband' : frequency_band,
                                           'radius' : transducer_radius,
                                           'thickness_td' : transducer_thickness_1,
                                           'thickness_el' : electrode_thickness,
                                           'thickness_sub' : substrate_thickness,
                                           'Tload' : top_load,
                                           'Telectrode' : electrode_mat,
                                           'piezo' : piezo_mat,
                                           'Belectrode' : electrode_mat,
                                           'Bsubstrate' : substrate_mat,
                                           'Bload' : bottom_load},
                               matpath=mat_path)

Simulation_2 = simulate_xMason(parameters={'fband' : frequency_band,
                                           'radius' : transducer_radius,
                                           'thickness_td' : transducer_thickness_2,
                                           'thickness_el' : electrode_thickness,
                                           'thickness_sub' : substrate_thickness,
                                           'Tload' : top_load,
                                           'Telectrode' : electrode_mat,
                                           'piezo' : piezo_mat,
                                           'Belectrode' : electrode_mat,
                                           'Bsubstrate' : substrate_mat,
                                           'Bload' : bottom_load},
                               matpath=mat_path)

In [ ]:
# Load experimental dats collected on 6 printed P(VDF-TrFE) samples

ISAF22_dataset = VNA_Dataloader(ISAF22_VNAexperiments, process=1, fcut=frequency_band[1])
samples = ['Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6']

In [ ]:
# Plot results

colors_red = sns.color_palette('Reds')
colors_blue = sns.color_palette('Blues')

fig = plt.figure(figsize=(5,4), dpi=250)
ax_1 = fig.add_subplot(111)
for idx in range(ISAF22_dataset.experiments.shape[2]-3):
    ax_1.plot((ISAF22_dataset.experiments[:, 0, idx] / 10**6), ISAF22_dataset.experiments[:, 6, idx],
              # markersize = 1,
              linewidth=2,
              color=colors_red[idx],
              alpha=0.4,
              label=samples[idx])

for idx in range(ISAF22_dataset.experiments.shape[2]-3):
    idx = idx+3
    ax_1.plot((ISAF22_dataset.experiments[:, 0, idx] / 10**6), ISAF22_dataset.experiments[:, 6, idx],
              # markersize = 1,
              linewidth=2,
              color=colors_blue[idx],
              alpha=0.4,
              label=samples[idx])

ax_1.plot((frequency_spectrum(frequency_band) / 10 ** 6), np.abs(Simulation_1.impedance.electric_impedance),
          markersize=1,
          linewidth=3,
          color=colors_blue[5],
          label=f'Simulation 1')

ax_1.plot((frequency_spectrum(frequency_band) / 10 ** 6), np.abs(Simulation_2.impedance.electric_impedance),
          markersize=1,
          linewidth=3,
          color=colors_red[5],
          label=f'Simulation 2')


ax_1.legend(ncol=3, loc="upper right", fontsize=6)
ax_1.set_yscale('log')
ax_1.set_xlabel('Frequency [MHz]', fontsize=18)
ax_1.set_ylabel(r'$\mid Z\mid [\Omega]$', fontsize=18)
ax_1.tick_params(axis='x', labelsize=15)
ax_1.tick_params(axis='y', labelsize=15)

plt.tight_layout()
plt.show()
